In [1]:
import os
import requests
import asyncio
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from IPython.display import clear_output
from systemlink.clients.nitag.api.tags_api import TagsApi

In [2]:
api_key = os.getenv("SYSTEMLINK_API_KEY")
http_url = os.getenv('SYSTEMLINK_HTTP_URI')
os.environ['SKYLINE_USER_SERVICES_URL'] = http_url
create_tag_selection_endpoint = f"{http_url}/nitag/v2/selections"
delete_tags_in_selection = f"{http_url}/nitag/v2/update-tags"
auth_api_key_endpoint = f"{http_url}/niauth/v1/auth"

In [3]:
session = requests.Session()
session.headers.update({
    "Content-Type": "application/json",
    'x-ni-api-key': api_key,
})
 
retry_strategy = Retry(
    total=3,  # total retry attempts
    status_forcelist=[500, 502, 503, 504],  # which errors to retry
    allowed_methods=["HEAD", "GET", "OPTIONS", "POST", "PUT", "DELETE"],  # methods to retry
    backoff_factor=2  # wait 1s, 2s, 4s ... between retries
)

# Mount the retry strategy to http and https
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)
 

## Get Workspace IDs

In [4]:
def get_workspace_ids():
    response = session.get(url=auth_api_key_endpoint)
    response.raise_for_status()
    data = response.json()
    
    workspaces = data["workspaces"]

    enabled = [workspace['id'] for workspace in workspaces if workspace["enabled"] == True]
    return enabled
    return [workspace['id'] for workspace in workspaces]

## Create selection for faster delete tags

In [5]:
def create_selection(workspace_id):
    try:
        payload = {
            "searchPaths": [
                "*Utilization*"
            ],
            "inactivityTimeout": 900,
            "workspace": workspace_id
        }
        resp = session.post(url=create_tag_selection_endpoint, json=payload)
        resp.raise_for_status()
        data = resp.json()
    
        return data['id']
    except Exception as e:
        print('workspace_id', workspace_id)
        print(e)
        return False

## Delete tags in selection

In [6]:
def delete_selection(selection_id):
    try:
        resp = session.delete(url=f"{http_url}/nitag/v2/selections/{selection_id}/tags")
        resp.raise_for_status()
        if resp.status_code == 204:
            return True
    except Exception as e:
        print('selection_id', selection_id)
        print(e)
        return False

In [7]:
workspace_ids = get_workspace_ids()

for ws_id in workspace_ids:
    selection_id = create_selection(ws_id)
    if not selection_id:
        continue
    deleted = delete_selection(selection_id)